# Train Custom Classifier

In [1]:
from google.colab import drive, files
drive.mount('/content/drive/')
path = "/content/drive/MyDrive/ANN-Project/"

Mounted at /content/drive/


In [2]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.nn.functional as F
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import pandas as pd
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

In [3]:
imported_weights = torch.load(path+'layer_3_output_weights_tensor.pth')
imported_bias = torch.load(path+'layer_3_output_bias_tensor.pth')
print(imported_weights)
print(imported_bias)

Parameter containing:
tensor([[-0.0147, -0.0082,  0.0009,  ..., -0.0163,  0.0048, -0.0019],
        [ 0.0096, -0.0054,  0.0079,  ..., -0.0095, -0.0122, -0.0027],
        [-0.0088, -0.0098, -0.0210,  ..., -0.0142, -0.0027, -0.0104],
        ...,
        [ 0.0377, -0.0108,  0.0385,  ..., -0.0146,  0.0211, -0.0067],
        [ 0.0171, -0.0198,  0.0186,  ..., -0.0051,  0.0050, -0.0094],
        [ 0.0110, -0.0172,  0.0157,  ...,  0.0113,  0.0073, -0.0129]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0077, -0.0071, -0.0068,  ...,  0.0499,  0.0134,  0.0499],
       requires_grad=True)


In [ ]:
#df = pd.read_csv(path+'dataset.csv')
#df.describe()

In [4]:
class Tekken3PoseDataset(Dataset):
    def __init__(self, csv_file):
        self.annotations = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        pose_data = self.annotations.iloc[index, 2:]
        pose_data.fillna(0, inplace=True)
        pose_data = torch.tensor(pose_data).float()
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        return (pose_data, y_label)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# Load Data
dataset = Tekken3PoseDataset(
    csv_file=path+"dataset.csv"
)

# 70:30 -> 4285

train_set, test_set = torch.utils.data.random_split(dataset, [3000, 1285])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First hidden layer (fc)
      self.h1 = nn.Linear(225, 4096)
      # Second hidden layer (fc)
      self.h2 = nn.Linear(4096, 4096)
      #  hidden layer that outputs our 2 labels
      self.output = nn.Linear(4096, 2)
#      self.dropout = nn.Dropout(p=0.4)

    def forward(self, x):
      x = nn.functional.relu(self.h1(x))
      x = nn.functional.relu(self.h2(x))
#      x = self.dropout(nn.functional.relu(self.h2(x)))
      x = nn.functional.softmax(self.output(x))
      return x

model = Net()

model.h2.weight.data = imported_weights
model.h2.bias.data = imported_bias

model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Cost at epoch 0 is 0.8272242653877178
Cost at epoch 1 is 0.8271134432325972
Cost at epoch 2 is 0.827445886870648
Cost at epoch 3 is 0.827113447037149
Cost at epoch 4 is 0.8275567077575846
Cost at epoch 5 is 0.8270026394661437
Cost at epoch 6 is 0.8273350786655507
Cost at epoch 7 is 0.8271134425985053
Cost at epoch 8 is 0.8270026274183964
Cost at epoch 9 is 0.8270026312229481
Checking accuracy on Training Set
Got 1458 / 3000 with accuracy 48.60
Checking accuracy on Test Set
Got 635 / 1285 with accuracy 49.42
